# Recommender with SVD and Reinforcement Learning

* __Background:__ Consider the following situation. We want to recommend items to users. The data we have about the user and item is summarized in a matrix $X$. It has $N$ rows representing users and $M$ columns representing items. The value in the n-th row and m-th column has 3 possible values: i) 1, if the n-th user has purchases the m-th item. ii) -1, if the n-th user has seen but did not purchase the m-th item. iii) 0, if the n-th user has not seen the m-th item before. A recommender would use these known information to guess what next item each user may like to buy and recommend that item. Such a recommender is **myopic** before it cares about the immediate reward, $R_{\mathrm{t+1}}$, i.e. when the recommender recommends an item at time step $t$, it cares about whehter the user would buy this item at time step $t+1$.


* __Goal:__ We would like to enable a SVD recommender to care about long-term reward by taking use of reinforcement learning (RL). In other words, we want to use RL algorithm to help an SVD recommender to care about the long-term return defined as $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...$, where $\gamma\in [0,1]$ is the discounting factor. Reinforcement Learning (RL) usually contains two major parts. First, policy evaluation (prediction problem), meaning to estimate the value function for a policy. In theory, the value function is the expected value for the future return, i.e. $G_t$. Second, policy improvement (control problem), meaning to improve the policy so that its value increases. In most cases, policy improvement must be done according the policy's value function. Therefore, the first part could be done on its own without the second part, but the second part must be done together with the first part. 


* __Methods:__  

    * _The myopic (greedy) recommender_ is based on ["Truncated SVD"](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html), which is a popular method in the category of [collaborative filtering](https://heartbeat.fritz.ai/recommender-systems-with-python-part-iii-collaborative-filtering-singular-value-decomposition-5b5dcb3f242b) recommendation. "Collaborative filtering" assumes that users who share similar preferences in the past will continue to like similar items in the future. 
    * The algorithm for _policy evaluation_ is "Semi-gradient TD(0)" described in Chapter 9 in the Book ["Reinforcement Learning: An Introduction" by Richard S. Sutton and Andrew G. Barto](http://incompleteideas.net/book/bookdraft2017nov5.pdf). The algorithm for _policy improvement_ will be discussed in details later in the Section on "Policy Improvement" in this notebook. This is a simple and naive idea of mine and I am not sure whether it would work out. I guess this simple idea probably has been investigated by other people before. I don't dive into literature for now and this notebook is not meant for publication for now. 
    * The interaction between users and the recommender would be simulated by a _Monte Carlo Simulator_.



## Initialization

Let us import libaries, define some parameters and initialize some variables.

In [1]:
import numpy as np
import torch
if torch.cuda.is_available():
    print("cuda is available")
    import torch.cuda as t
    device = torch.device('cuda:0')
else:
    print("cuda is unavailable")
    import torch as t
    device = torch.device('cpu')

cuda is unavailable


* Define the ground-truth `Xtrue` containing the true preferences of users to items. The Monte-Carlo simulator knows `Xtrue` in order to simulate the user's response to the recommender's recommendations. The recommender of course does not know `Xtrue`.

* Initialize `X`, by revealing each user's preferences to `M_start` items randomly. The recommender could only see `X` which contains users' preferences to items that have been shown to them.

* Define two functions: one for creating polynomial feature vector with power 2, the other is to choose an un-seen item in each row of `X`.

In [2]:
# --------- Define Parameters ----------------
N = 1000 # number of users
M = 100 # number of items
K = 10 # number of singular values (eigen values) to retain in Truncated SVD.
power = 2 # the power for polynomial feature vector, e.g. [x1, x2] --> [1, x1, x2, x1**2, x1*x2, x2**2]
eps = 0.1 # probability for choosing the random action
eta = 0.0 # improvement rate
alpha = 0.3 # learning rate for Reinforcement learning v <--- (1-alpha)* v + alpha * target_v
gamma = 0.3 # discounting factor for future return, return_t = immediate_reward + gamma * return_{t+1}

# Initialize the weights in action-value function, Wa, to all zeros.
n_S = K * (N + M + 1)
n_SA = K * (N + M + 1) + N 
len_Ws = int(n_S**2/2 + 3*n_S/2 + 1)
len_Wa = int(n_SA**2/2 + 3*n_SA/2 + 1)
Ws = torch.zeros(len_Ws, device=device)
Wa = torch.zeros(len_Wa, device=device)

# `Xtrue` is the ground-truth matrix for all users' preferences to all items. 
# It is known by the Monte-Carlo Simulator but not known by the recommender.
# `Xtrue` is generated randomly such that its element is 1 by 50% chance and 
# -1 by 50% chance.
Xtrue = torch.rand(N, M, device=device)
Xtrue[Xtrue >= 0.5] = 1 
Xtrue[Xtrue < 0.5] = -1

# In real life, a recommender faces a continuing task. For simplicity of this project, we 
# artificially define an "episode" during which the recommender interacts with users 
# by M_recom times.
M_recom = int(0.2 * M)

# We initialize X by revealing the ground-truth preferences to M_start items for each user
M_start = int(0.1 * M)

# Initialize X
X = torch.zeros(N, M, device=device)
X_now = X

for i in range(0, N):
    rand_index = np.random.choice(range(0,M), size=M_start, replace=False)
    X[i, rand_index] = Xtrue[i, rand_index]
    

# A function creating polynomial feature vector of power 2.
def poly2_features(x_vector):
    '''
    Map original features to its polynomial feature vector with power=2 and 
    normalize the resulting feature vector so that its L2-norm = 1, e.g.
    [x1, x2, x3] ---> pre_vector = [1, x1, x2, x3, x1*x2, x2*x3, x1*x3, x1^2, x2^2, x3^2], 
    and then map the pre_vector to: pre_vector/(L2-norm of pre_vector)
    
    Input:
    ------
    x_vector: 2D torch tensor of shape ([1, num_columns])
              containing the orginal features. 
    
    Output:
    ------
    x_poly2: 1D torch vector of shape ([length]) 
             containing the polynomial features with power 2. 
    '''
    x_2 = x_vector.reshape(-1,1) * x_vector
    num_columns = x_vector.shape[1]
    x_2 = x_2[np.triu_indices(num_columns)]
    x_poly2_pre = torch.cat([t.FloatTensor([1]), x_vector.reshape(-1), x_2], dim=0)
    x_poly2 = x_poly2_pre / torch.norm(x_poly2_pre)
    
    return x_poly2

# Pick an unseen item randomly for each user (row).
def rand_unseen(row):
    return np.random.choice(np.argwhere(row == 0).reshape(-1), 1)

## Establish the Policy

* __Truncated SVD:__ One way for [collaborative filtering](https://heartbeat.fritz.ai/recommender-systems-with-python-part-iii-collaborative-filtering-singular-value-decomposition-5b5dcb3f242b) recommendation is to use ["Truncated SVD"](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) to decompose the matrix $X$ and reconstruct this matrix's approximation $\tilde X$. "Truncated" here means that we only retain the first $K \ll \mathrm{min}(N, M)$ largest eigen values for $X^\mathrm{T}X$ in the process of Singular Value Decomposition (SVD). The zeros in the original $X$ would be mostly filled with non-zero values in the approximation $\tilde X$. These re-filled non-zeros values (which were zero before) can be regarded as the predicted "probability" for the user to purchase this new item. An SVD recommender would recommend the item with the highest re-filled value for each user. I see `torch` does not have TrunactedSVD yet. So I use the one in `sklearn`.


* __$\epsilon$-greedy policy__: The svd-action is based on the "Truncated SVD" briefly described above. We will use the action-value function (to be evaluated later) to make some improvement the "Truncated SVD" action. The improved version is the greedy action. The policy adopts the greedy action most of the time (by probability $1-\epsilon$), and recommends a item among the un-seen items randomly for the rest of the time (by probability $\epsilon \ll 1$). Given that the data matrix $X$ is usually very sparse, i.e. the recommender knows only a little about each user's preference to items, the predicted "probability" for purchase cannot be correct all the time. So it makes sense that for a small amount of time, we just recommend random items to the users. The $\epsilon$ part of the policy allows us to explore "what is unknown by the recommender".

### Truncated SVD

* Truncated SVD on `X` leading to the matrix `U`, `Sigma`, and `Vt`. 

* Refill (or re-construct) `X` and assign the refilled version to `X_refilled`. `X_refilled` contains predicted preferences to un-seen items. 

* Find the un-seen item with the max preference predicted by the Truncated SVD method for each user (row in `X_refilled`), and assign the indices of those items to `A_greedy`. `A_greedy` is named in this way meaning "greedy action" which exploits the known knowledge greedily.

In [3]:
n_iter = 10 * K # number of iterations for Truncated SVD.

# If you need U, Sigma, Vt directly, please call the following two lines.
from sklearn.utils.extmath import randomized_svd
U, Sigma, Vt = randomized_svd(X.numpy(), n_components=K, n_iter=n_iter)
X_refilled = np.matmul(np.matmul(U, np.diag(Sigma)), Vt)

# Assign very negative number to items that have been seen by users.
# So that they won't be picked by the recommender.
X_refilled[X != 0] = -1e3

# find the index of the item with the highest predicted preference for each user (row)
A_svd = np.apply_along_axis(np.argmax, 1, X_refilled)

###  Improve the Action (Policy Improvement) 

We now have found the svd action based on the Truncated SVD. We like to improve this greedy action based on the action-value function to be learned by Reinforcement Learning, $q(s,a)$.

* Use `U`, `Sigma` and `Vt` to compose the state vector, `S_now`, which will be used to calculate the value functions, which is the essential part of reinforcement learning.


* Use the approximate action-value function, $q(s, a) = W_a^\mathrm{T} \mathbf{x}(s,a)$, to improve the greedy action. Here $\mathrm{x}(s,a)$ is the polynomial feature vector for the state and action, which we call `S_A_poly2` in the cell below. Note that $q$ is linear in $W_a$. The weights for $q$, $W_a$, will be iteratively learned by "Semi-gradient TD(0)" algorithm later. This algorithm is described in Chapter 9 in the book "Reinforcement Learning: An Introduction" by Sutton and Barto. I adopt the following assignment for the __policy improvement__:

    $A_\mathrm{greedy} \leftarrow A_\mathrm{greedy} + \eta {\partial q(S_\mathrm{now}, A_\mathrm{greedy})\over \partial A_\mathrm{greedy}}\, ,$
    
    where $\eta$ is a positive number which I call "improvement rate".
    
__Note:__ Usually policy improvement means to find the action that maximizes the action-value function, meaning that $A_\mathrm{greedy} \leftarrow \mathrm{argmax}_{a} q(S_\mathrm{now}, a)$. However, this is not practical. Therefore, I choose to first find the action based on Truncated-SVD and then use the action-value function to make a small improvement on the Truncated-SVD action. It is a "small improvement" because $0 < \eta <1$ (small) and this change to the action would make the $q$ a bit greater (improvement).

In [4]:
# scale Sigma
Sigma_scaled = (Sigma - Sigma.min()) / (Sigma.max() - Sigma.min())

# Represent the current state by S_now
S_now = np.concatenate([U.reshape(-1), Sigma_scaled, Vt.reshape(-1)], axis=0)
S_now = t.FloatTensor(S_now).requires_grad_(True)


# I need dq/dA_greedy, so I shall define A_greedy as a torch.tensor with
# requires_grad=True.
A_svd = t.FloatTensor(A_svd).requires_grad_(True)

# Scaled A_greedy
A_svd_scaled = A_svd/M

# Put S_now and A_greedy_scaled into a 1D torch.Tensor
S_A = torch.cat([S_now.reshape(1,-1), A_svd_scaled.reshape(1,-1)], dim=1)

# Construct a polynomial Feature vector with power 2 for the state and action pair, (S_now, A_greedy)
S_A_poly2 = poly2_features(S_A)

# Calculate action-value q(S_now, A_greedy)
q_Asvd = torch.matmul(Wa.reshape(1,-1), S_A_poly2.reshape(-1,1)) 

# This could only be done once, once it is done, the graph leading to q_Agreedy is destroyed.
q_Asvd.backward()

# dq/dAgreedy, A_greedy.grad could only be called once as well!
dq_dAsvd = A_svd.grad

# Improve A_greedy, remember to make sure A_greedy contains integers
A_greedy = torch.round(A_svd + eta * dq_dAsvd).long()

### Random Action

Recall that we decide to adopt an $\epsilon$-greedy policy, meaning that for a small amount of time we take a random action (by probabily $\epsilon$) in order to explore what is unknown to the recommender. We have found and improved the greedy action, `A_greedy`. Now let us randomly pick an un-seen item for each user, which is the random action.

In [5]:
# This is to make sure that A_rand has the same type as A_greedy
A_rand = t.IntTensor(np.apply_along_axis(rand_unseen, 1, X).reshape(-1))

### $\epsilon$-Greedy Policy

* Choose the greedy action, `A_greedy`, by the probability $1-\epsilon$.

* Choose the random action, `A_rand`, by the probability $\epsilon$.

In [6]:
# A_now is the current action taken for the current state S_now
# A_now follows an epsilon-greedy policy

choose_greedy = np.random.choice([True, False], 1, [1e0-eps, eps])
# The unscaled A_now is for updating the next X, since A_now contains the indices for items to recommend.
if choose_greedy:
    A_now = A_greedy
else:
    A_now = A_rand
    
# Scale the action vector, A_now, this scaled version is for calculating value functions
A_now_scaled = A_now.float() / M

### Design a Class "Policy".

Now it is time to wrap up the sections on __"Truncated SVD", "Improve the Action", "Random Action", "$\epsilon$-Greedy Policy"__ described above into a Class `Policy`. This Class should map `X` (users' preferences to part of items, visible to the recommender) to action `A` using the $\epsilon$-greedy policy. It has the following attributes and handy methods. The symbol denoting a such a map is conventionally $\pi : s \rightarrow a$, mapping state to an action.

* __Attributes:__ 
    * `X`: input 2D matrix containing part of users' preferences.
    * `eps`: the value of epsilon for the $\epsilon$-greedy policy.
    * `eta`: improvement rate.
    * `Wa`: the weights in the action-value function.
    * `only_svd`: True if taking the action based on Truncated SVD, False if taking the action based on SVD improved with RL.
    * `K`: the number of sigular values to retain in the Truncated SVD algorithm.
    * `N`: the number of rows in `X`, equal to the number of users.
    * `M`: the number of columns in `X`, equal to the number of items.
    * `S`: 1D vector containing elements in the matrices, `U`, `Sigma`, `Vt`, resulting from the Truncated SVD of `X`. This 1D vector represents the current state approximately. Its length is `len(S)` $= K(N+M+1)$.
    * `A`: 1D int vector containing the indices of items to be recommended to users, i.e. $m(n)$ for $n=0,..., N-1$. So its length is `len(A)` $= N$.
 

* __Methods must be called:__
    * .fill_S(self): fill the attribute `self.S`.
    * .fill_A(self): fill the attribute `self.A`. Choose one from `A_greedy` and `A_rand` as the action to be taken with probabilities of $1-\epsilon$ and $\epsilon$ respectively.
    
    You must call both `.fill_S` and `.fill_A` after you define object in the Class `Policy`. Otherwise, the attributes `Policy.S` and `Policy.A` would be all zeros.
    
    
* __Other Methods:__
    * .get_U_Sigma_Vt(self): get the matrix factorizations due to Truncated SVD.
    * .get_Asvd(self): get the recommendation vector `Asvd` based on the Truncated SVD algorithm.
    * .get_Agreedy(self, Wa): get the improved action, `Agreedy`, based on `Asvd` and the action-value function's weights `Wa`.
    * .get_Arand(self): get the random action, meaning randomly recommend one un-seen item to each user.
    * .get_state_poly(self): get the polynomial feature vector representing the state.
    * .get_state_action_poly(self): get the polynomial feature vector representing the state and the action.

In [7]:
class Policy():
    '''
    Policy, mappying state to action.
    '''
    def __init__(self, X, K, eps, eta, Wa, only_svd=False):
        self.X = X
        self.K = K
        self.eps = eps
        self.eta = eta
        self.Wa = Wa
        self.only_svd = only_svd
        self.N = X.shape[0]
        self.M = X.shape[1]
        self.S = t.zeros(K * (self.N + self.M +1))
        self.A = t.zeros(self.N)
        
    def get_U_Sigma_Vt(self):
        '''
        Get the matrix factorizations due to Truncated SVD.
        '''
        n_iter = 10 * self.K # number of iterations for Truncated SVD.

        # Truncated SVD: X ---> U, Sigma, Vt ---> X_refilled
        from sklearn.utils.extmath import randomized_svd
        U, Sigma, Vt = randomized_svd(self.X.numpy(), n_components=self.K, n_iter=n_iter)
        
        return U, Sigma, Vt
        
    def get_Asvd(self):
        '''
        Get the action due to Truncated SVD algorithm
        '''
        U, Sigma, Vt = self.get_U_Sigma_Vt()
        X_refilled = np.matmul(np.matmul(U, np.diag(Sigma)), Vt)

        # Assign very negative number to items that have been seen by users.
        # So that they won't be picked by the recommender.
        X_refilled[self.X != 0] = -1e3

        # find the index of the item with the highest predicted preference for each user (row)
        A_svd = np.apply_along_axis(np.argmax, 1, X_refilled)
        
        return A_svd
    
    def fill_S(self):
        '''
        Fill the attribute self.S.
        '''        
        # call get_Asvd
        U, Sigma, Vt = self.get_U_Sigma_Vt()
        
        # scale Sigma
        Sigma_scaled = (Sigma - Sigma.min()) / (Sigma.max() - Sigma.min())

        # Represent the current state by S_now
        S_now = np.concatenate([U.reshape(-1), Sigma_scaled, Vt.reshape(-1)], axis=0)
        self.S = t.FloatTensor(S_now).requires_grad_(True)
    
    def get_Agreedy(self):
        '''
        Get the greedy action, an improved version of A_svd based on the action-value function.
        '''
        # Call get_S
        A_svd = self.get_Asvd()

        # I need dq/dA_greedy, so I shall define A_greedy as a torch.tensor with
        # requires_grad=True.
        A_svd = t.FloatTensor(A_svd).requires_grad_(True)

        # Scaled A_greedy
        A_svd_scaled = A_svd / self.M

        # Put S_now and A_greedy_scaled into a 1D torch.Tensor
        S_A = torch.cat([self.S.reshape(1,-1), A_svd_scaled.reshape(1,-1)], dim=1)

        # Construct a polynomial Feature vector with power 2 for the state and action pair, (S_now, A_greedy)
        S_A_poly2 = poly2_features(S_A)

        # Calculate action-value q(S_now, A_greedy)
        q_Asvd = torch.matmul(self.Wa.reshape(1,-1), S_A_poly2.reshape(-1,1)) 

        # This could only be done once, once it is done, the graph leading to q_Agreedy is destroyed.
        q_Asvd.backward()

        # dq/dAgreedy, A_greedy.grad could only be called once as well!
        dq_dAsvd = A_svd.grad

        # Improve A_greedy, remember to make sure A_greedy contains integers
        A_greedy = torch.round(A_svd + self.eta * dq_dAsvd).long()
        
        return A_greedy
    
    def get_Arand(self):
        '''
        Get the random action.
        '''
        # This is to make sure that A_rand has the same type as A_greedy
        A_rand = t.IntTensor(np.apply_along_axis(rand_unseen, 1, self.X).reshape(-1))
        
        return A_rand
    
    def fill_A(self):
        '''
        Choose one action from A_greedy and A_rand by chances 1-eps and eps respectively.
        '''
        if self.only_svd:
            A_svd = self.get_Asvd()
            self.A = t.IntTensor(A_svd).reshape(-1)
        else:
            A_greedy = self.get_Agreedy()
            A_rand = self.get_Arand()

            choose_greedy = np.random.choice([True, False], 1, [1e0-self.eps, self.eps])
            # The unscaled A_now is for updating the next X, 
            # since A_now contains the indices for items to recommend.
            if choose_greedy:
                A_now = A_greedy
            else:
                A_now = A_rand

            # Assign the attribute
            self.A = A_now
    
    def get_state_poly(self):
        '''
        Get the polynomial feature with power=2 for the state.
        '''
        S_poly2 = poly2_features(self.S.reshape(1,-1))
        
        return S_poly2
    
    def get_state_action_poly(self):
        '''
        Get the polynomial feature with power=2 for the (state, action) pair.
        '''
        A_now_scaled = self.A.float() / self.M
        
        # Put S_now and A_greedy_scaled into a 1D torch.Tensor
        S_A = torch.cat([self.S.reshape(1,-1), A_now_scaled.reshape(1,-1)], dim=1)

        # Construct a polynomial Feature vector with power 2 for the state and action pair
        S_A_poly2 = poly2_features(S_A)
        
        return S_A_poly2

In [8]:
Pi = Policy(X_now, K, eps, eta, Wa)
Pi.fill_S()
Pi.fill_A()

A_now = Pi.A
S_poly2 = Pi.get_state_poly()
S_A_poly2 = Pi.get_state_action_poly()

### Observe the Next State, the Immediate Reward. 

* __Next state:__ Once the recommender recommends one item to each user, the users would react by "purchasing" (+1) or "not purchasing" (-1). The ground-truth user's preferences are stored in the matrix `Xtrue`. For example, the recommender recommends the m-th item to the n-th user, then we would assign the n-th row & m-th column element in `X` 
to that element in `Xtrue` at the same position. After updating `X`, we could use it to compose the next state, `S_next`.


* __Immediate reward:__ We care about the average (across all users) user's satisfaction of our recommendations. So I define the averaged true preference to the recommended item across all users as the immediate reward, `R_next`.



In [9]:
# Xtrue, Xnow, Anow ---> Xnext, Rnext
def get_next(Xtrue, X_now, A_now):
    '''
    Get the next state and reward given the current state, action and ground-truth state
    
    Input:
    -----
    Xtrue: 2D int matrix, the ground-true preferences of users to items.
    X_now: 2D int matrix, the current revealed preferences of users to items.
    A_now: 1D int vector, the indices of items to be recommended to all users.
    
    Output:
    -------
    X_next: 2D int matrix, adding users' preferences to newly recommended items to X_now.
    R_next: Float, the mean user's preference to the recommended items.
    '''
    
    N = Xtrue.shape[0] # number of rows (users)
    reveal_indices = (np.arange(0,N), np.array(A_now))
    X_next = X_now
    X_next[reveal_indices] = Xtrue[reveal_indices]
    R_next = Xtrue[reveal_indices].sum() / N
    return X_next, R_next

Get the next state matrix `X_next` and the immediate reward `R_next`.

In [10]:
X_next, R_next = get_next(Xtrue, X_now, A_now)

Calculate the polynomial feature vectors for the next state and the next (state, action) pair.

In [11]:
Pi_next = Policy(X_next, K, eps, eta, Wa)
Pi_next.fill_S()
Pi_next.fill_A()

S_poly2_next = Pi_next.get_state_poly()
S_A_poly2_next = Pi_next.get_state_action_poly()

## Policy Evaluation (Prediction Problem)

I adopt the "Semi-gradient TD(0)", specifically Equation (9.9), in the Book ["Reinforcement Learning: An Introduction" by Richard S. Sutton and Andrew G. Barto](http://incompleteideas.net/book/bookdraft2017nov5.pdf). I copy this equation below.

$ \mathbf{w}_{t+1} \leftarrow \mathrm{w}_t + \alpha\left(R_{t+1} \mathbf{x}_t - \mathrm{x}_t(\mathrm{x}_t-\gamma\mathrm{x}_{t+1})^\mathrm{T} \mathrm{w}_t\right)$,

where the feature vector $\mathrm{x}_t$ in our case is the polynomial feature vector with power 2. This formula applies to both the weight vector for the state-value function

$v(s) = \mathrm{W}_s^\mathrm{T} \mathrm{x}(s)$ ,

and the action-value function,

$q(s, a) = \mathrm{W}_a^\mathrm{T} \mathrm{x}(s, a)$ .

When we update the weight vector `Ws`, we make the following replacements to Equation (9.9):

* $\mathrm{x}_t\rightarrow$ `S_poly2`
* $\mathrm{x}_{t+1}\rightarrow$ `S_poly2_next`
* $\mathrm{w}_{t+1}\, , \mathrm{w}_t\rightarrow$ `Ws`.
* $R_{t+1}\rightarrow$ `R_next`.

When we update the weight vector `Wa`, we make the following replacements to Equation (9.9):

* $\mathrm{x}_t\rightarrow$ `S_A_poly2`
* $\mathrm{x}_{t+1}\rightarrow$ `S_A_poly2_next`
* $\mathrm{w}_{t+1}\, , \mathrm{w}_t\rightarrow$ `Wa`.
* $R_{t+1}\rightarrow$ `R_next`.

The cell below could only be run once per time step in the Monte Carlo simulation.

In [12]:
# Update Ws, weights for the state-value function.
Ws_Delta_S = torch.mm((S_poly2 - gamma*S_poly2_next).reshape(1,-1), Ws.reshape(-1,1)).reshape(-1)
Ws = Ws + alpha * (R_next - Ws_Delta_S) * S_poly2

# Update Wa, weights for the action-value function.
Wa_Delta_SA = torch.mm((S_A_poly2 - gamma*S_A_poly2_next).reshape(1,-1), Wa.reshape(-1,1)).reshape(-1)
Wa = Wa + alpha * (R_next - Wa_Delta_SA) * S_A_poly2

Do not forget to update `X_now` when iterating a number of recommendations.

In [13]:
X_now = X_next

## Wrapping Up

Now it is time to wrap up everything in this notebook into two python scripts:

* "workhorse" script, named "utils.py", defining functions and classes.
* "simulation" script, carrying out the Monte Carlo simulations on the interactions between users and the recommender. During each episode of the Monte Carlo simulation (e.g. 30 times of interactions), the state and action value functions would be learned and the policy would be improved according to the action-value function. One interaction refers to the process that the recommender recommends one item to each user and every user responds by either purchasing or not purchasing. The "simulation.py" script runs the episode until the total number of episodes reaches a pre-set maximum number or the weights for the value functions converge to a pre-set level.